In [ ]:
# import the packages we need
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from shapely.geometry import Point
from datetime import datetime
from pyproj import CRS
import numpy as np
import plotly.express as px
from datetime import timedelta
import os
import json

### POI cleaning

In [ ]:
# filter out low quality (less than 3 not contained) poi data
poi=poi[poi.QUALITY>=3]
# drop useless columns
poi.drop(columns=['QUALITY'],inplace=True)

In [ ]:
# extract level1 category code
poi['level1']=poi['CAT_CODE']
poi['level1']= [x[:3] for x in poi['CAT_CODE']]
poi['level1'] = poi['level1'].astype(str).astype(int)
poi.head()

In [ ]:
# clean poi based on chain name
poi_1= poi[(poi['level1']==400)|(poi['level1']==550)|(poi['level1']==900)|(poi['level1']==350)|(poi['level1']==800)]
poi_2= poi[(poi['level1']!=400)&(poi['level1']!=550)&(poi['level1']!=900)&(poi['level1']!=350)&(poi['level1']!=800)].dropna(subset=['CHAIN_NAME'])
poi1=pd.concat([poi_1,poi_2])

### POI opening and closing hour

opening hour and closing hour are in json format. the following code attempts to transfrom json format to readable format


In [ ]:
# deal with json format
opening_hour['LABELS'] = opening_hour['JSON_OPENINGTIME'].str.strip('[]').str.split(', ')
opening_hour.head(3)

In [ ]:
# extract opening and closing time to columns
opening_hour['SUNDAY_O']='0'
opening_hour['SUNDAY_C']='0'

opening_hour['SATURDAY_O']='0'
opening_hour['SATURDAY_C']='0'

opening_hour['FRIDAY_O']='0'
opening_hour['FRIDAY_C']='0'

opening_hour['THURSDAY_O']='0'
opening_hour['THURSDAY_C']='0'

opening_hour['WEDNESDAY_O']='0'
opening_hour['WEDNESDAY_C']='0'

opening_hour['TUESDAY_O']='0'
opening_hour['TUESDAY_C']='0'

opening_hour['MONDAY_O']='0'
opening_hour['MONDAY_C']='0'

for j in range(0,len(opening_hour.index)):
    for i in (0,3,6,9,12,15,18):
        try: 
            if opening_hour.LABELS[j][i][15:21]=='SUNDAY':
                opening_hour['SUNDAY_O'][j]=pd.to_numeric(opening_hour.LABELS[j][i+1][-9:-7])
                opening_hour['SUNDAY_C'][j]=pd.to_numeric(opening_hour.LABELS[j][i+2][-10:-8])
            if opening_hour.LABELS[j][i][15:23]=='SATURDAY':
                opening_hour['SATURDAY_O'][j]=pd.to_numeric(opening_hour.LABELS[j][i+1][-9:-7])
                opening_hour['SATURDAY_C'][j]=pd.to_numeric(opening_hour.LABELS[j][i+2][-10:-8])
            if opening_hour.LABELS[j][i][15:21]=='FRIDAY':
                opening_hour['FRIDAY_O'][j]=pd.to_numeric(opening_hour.LABELS[j][i+1][-9:-7])
                opening_hour['FRIDAY_C'][j]=pd.to_numeric(opening_hour.LABELS[j][i+2][-10:-8])
            if opening_hour.LABELS[j][i][15:23]=='THURSDAY':
                opening_hour['THURSDAY_O'][j]=pd.to_numeric(opening_hour.LABELS[j][i+1][-9:-7])
                opening_hour['THURSDAY_C'][j]=pd.to_numeric(opening_hour.LABELS[j][i+2][-10:-8])
            if opening_hour.LABELS[j][i][15:24]=='WEDNESDAY':
                opening_hour['WEDNESDAY_O'][j]=pd.to_numeric(opening_hour.LABELS[j][i+1][-9:-7])
                opening_hour['WEDNESDAY_C'][j]=pd.to_numeric(opening_hour.LABELS[j][i+2][-10:-8])
            if opening_hour.LABELS[j][i][15:22]=='TUESDAY':
                opening_hour['TUESDAY_O'][j]=pd.to_numeric(opening_hour.LABELS[j][i+1][-9:-7])
                opening_hour['TUESDAY_C'][j]=pd.to_numeric(opening_hour.LABELS[j][i+2][-10:-8])
            if opening_hour.LABELS[j][i][15:21]=='MONDAY':
                opening_hour['MONDAY_O'][j]=pd.to_numeric(opening_hour.LABELS[j][i+1][-9:-7])
                opening_hour['MONDAY_C'][j]=pd.to_numeric(opening_hour.LABELS[j][i+2][-10:-8])
        except:
            continue


In [ ]:
# create a list of column names
col=list(opening_hour.columns.values)
col

In [ ]:
# convert str to int
for i in col[1:]:
    opening_hour[i]=opening_hour[i].astype(str).astype(int)

### merge the opening hour with POI data

In [ ]:
# merge with poi to get cat-code
poi_opening_hour = pd.merge(poi1, opening_hour,how="left", left_on='ID',right_on='POID')
poi_opening_hour

In [ ]:
# drop nan
poi_opening_hour= poi_opening_hour.dropna(subset=['NAME'])
poi_opening_hour

### median opening and closing hour for each type of poi

In [ ]:
# # assign category with median open and close hour
code=list(poi_opening_hour1.level1.unique())

for j in code:
    for i in col[1:]:
        opening = poi_opening_hour1[poi_opening_hour1.level1==j]
        opening = opening.dropna()
        try:
            n = round(opening[(opening[i] != 0)][i].median())
            categories.loc[categories.level1Code == j, i] = n
        
        except:
            categories.loc[categories.level1Code == j, i] = opening[(opening[i] != 0)][i].median()

categories


In [ ]:
# label the time segments
categories1=categories.copy()

for i in col[1:]:
    categories1.loc[categories1[i].isin(range(6,10)), i] = 'S1'
    categories1.loc[categories1[i].isin(range(10,13)), i] = 'S2'
    categories1.loc[categories1[i].isin(range(13,16)), i] = 'S3'
    categories1.loc[categories1[i].isin(range(16,19)), i] = 'S4'
    categories1.loc[categories1[i].isin(range(19,22)), i] = 'S5'
    categories1.loc[categories1[i].isin(range(22,25)), i] = 'S6'

categories1

In [ ]:
# merge with categories
poi_cat_hour = pd.merge(poi1, categories1, 
                            how="left", left_on='level1',right_on='level1Code')
poi_cat_hour